Earlier in class you were provided a design for the database (which can also be found in files/SQLProject/tables.docx)  and a set of data files (in files/data/SQLProject). You were asked to create a MySQL database using this design and data via a Python program (similar to creating the SQL HW DB).

In [53]:
# Connect to the database
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="421020"   
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [116]:
# Disconnect
mydb.close()
if not mydb.is_connected():
        print("CONNECTION CLOSED")

CONNECTION CLOSED


In [54]:
mycursor = mydb.cursor()

In [ ]:
# Create Database
mycursor.execute("CREATE DATABASE hospital_project")

In [7]:
mycursor.execute("SHOW DATABASEs")
print(mycursor.fetchall())

[('data1050F24',), ('hospital_project',), ('information_schema',), ('mysql',), ('performance_schema',), ('sys',)]


In [55]:
mycursor.execute("USE hospital_project")

In [ ]:
# create physicians table
mycursor.execute(
    """
    CREATE TABLE physicians (
        ssn VARCHAR(11) PRIMARY KEY,
        name VARCHAR(255),
        primary_specialty VARCHAR(255),
        experience_years INT
    )
    """
)

In [18]:
# create pharmacies table
mycursor.execute(
    """
    CREATE TABLE pharmacies (
        id INT PRIMARY KEY,
        name VARCHAR(255),
        address VARCHAR(255),
        phone VARCHAR(20)
    )
    """
)

In [ ]:
# create drug table
mycursor.execute(
    """
    CREATE TABLE drugs (
        drug_id INT,
        name VARCHAR(255) PRIMARY KEY
    )
    """
)

In [20]:
# create patient table
mycursor.execute(
    """
    CREATE TABLE patients (
        ssn VARCHAR(11) PRIMARY KEY,
        name VARCHAR(255),
        address VARCHAR(255),
        birth_date DATE,
        physician_id VARCHAR(11),
        FOREIGN KEY (physician_id) REFERENCES physicians(ssn)
    )
    """
)

In [ ]:
# create prescriptions table
mycursor.execute(
    """
    CREATE TABLE prescriptions (
        id INT PRIMARY KEY,
        patient_id VARCHAR(11),
        physician_id VARCHAR(11),
        drug_name VARCHAR(255),
        date DATE,
        quantity INT,
        FOREIGN KEY (patient_id) REFERENCES patients(ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians(ssn),
        FOREIGN KEY (drug_name) REFERENCES drugs(name)
    )
    """
)

In [75]:
mycursor.execute(
    """
    ALTER TABLE prescriptions
    ADD INDEX patient_drug_idx (patient_id, drug_name)
    """
)

In [ ]:
# create adverse interactions table
mycursor.execute(
    """
    CREATE TABLE adverse_interactions (
        drug_name_1 VARCHAR(255),
        drug_name_2 VARCHAR(255),
        PRIMARY KEY (drug_name_1, drug_name_2),
        FOREIGN KEY (drug_name_1) REFERENCES drugs(name),
        FOREIGN KEY (drug_name_2) REFERENCES drugs(name)
    )
    """
)

In [82]:
# create alerts table
mycursor.execute(
    """
    CREATE TABLE alerts (
        patient_id VARCHAR(11),
        physician_id VARCHAR(11),
        alert_date DATE,
        drug1 VARCHAR(255),
        drug2 VARCHAR(255),
        PRIMARY KEY (patient_id, physician_id, alert_date, drug1, drug2),
        FOREIGN KEY (patient_id) REFERENCES patients(ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians(ssn),
        FOREIGN KEY (patient_id, drug1) REFERENCES prescriptions(patient_id, drug_name),
        FOREIGN KEY (patient_id, drug2) REFERENCES prescriptions(patient_id, drug_name)
    )
    """
)

In [28]:
# create pharmacy_fills table
mycursor.execute(
    """
    CREATE TABLE pharmacy_fills (
        prescription_id INT,
        pharmacy_id INT,
        date DATE,
        cost DECIMAL(10, 2),
        PRIMARY KEY (prescription_id, pharmacy_id),
        FOREIGN KEY (prescription_id) REFERENCES prescriptions(id),
        FOREIGN KEY (pharmacy_id) REFERENCES pharmacies(id)
    )
    """
)

In [30]:
# create company table
mycursor.execute(
    """
    CREATE TABLE companies (
        id INT PRIMARY KEY,
        name VARCHAR(255),
        address VARCHAR(255),
        contact_phone VARCHAR(20),
        contact_name VARCHAR(255)
    )
    """
)

In [ ]:
# create contracts table
mycursor.execute(
    """
    CREATE TABLE contracts (
        id INT PRIMARY KEY,
        company_id INT,
        pharmacy_id INT,
        drug_name VARCHAR(255),
        dosage VARCHAR(255),
        quantity INT,
        date DATE,
        price DECIMAL(10, 2),
        FOREIGN KEY (company_id) REFERENCES companies(id),
        FOREIGN KEY (pharmacy_id) REFERENCES pharmacies(id),
        FOREIGN KEY (drug_name) REFERENCES drugs(name)
    )
    """
)

In [23]:
mycursor.execute("SHOW TABLES")
columns = mycursor.fetchall()
for column in columns:
    print(column)

('adverse_interactions',)
('alerts',)
('companies',)
('contracts',)
('drugs',)
('patients',)
('pharmacies',)
('pharmacy_fills',)
('physicians',)
('prescriptions',)


Here are some additional requirements we'd like to add to this database:

The hospital administration needs to retrieve from time to time primary specialty and years of experience of a given physician. Write a stored procedure that can be used to do that.

In [5]:
# create get_specialty_year procedure
mycursor.execute(
    """
    CREATE PROCEDURE get_specialty_year (
        IN physician_ssn VARCHAR(11),
        OUT p_specialty VARCHAR(255),
        OUT exp_years INT
    )
    BEGIN
        SELECT primary_specialty, experience_years
        INTO p_specialty, exp_years
        FROM physicians
        WHERE ssn = physician_ssn;
    END;
    """
)


In [5]:
mycursor.execute(
    """
    SELECT ROUTINE_NAME
    FROM information_schema.ROUTINES
    WHERE ROUTINE_SCHEMA = 'hospital_project'
    AND ROUTINE_NAME = 'get_specialty_year'
    AND ROUTINE_TYPE = 'PROCEDURE'
"""
)
result = mycursor.fetchone()
print(result)

('get_specialty_year',)


Write a trigger that inserts a tuple in the alerts table when a patient is prescribed a drug (i.e., after inserting a new row in the prescriptions table)  that adversely interacts with a drug the patient was earlier prescribed. So the event for the trigger is a patient being ascribed a drug (i.e., the insertion of a new row in the prescriptions table); the condition to be checked is whether the patient was earlier prescribed a drug that adversely interacts with the newly prescribed drug; the action is to insert a new tuple in the alerts table, where drug1 is the earlier prescribed drug and drug2 is the newly prescribed drug.

In [106]:
# create adverse_interaction_alert trigger
mycursor.execute(
    """
    CREATE TRIGGER adverse_interaction_alert
    AFTER INSERT ON prescriptions
    FOR EACH ROW
    BEGIN
        INSERT INTO alerts (patient_id, physician_id, alert_date, drug1, drug2)
        SELECT DISTINCT NEW.patient_id, NEW.physician_id, NEW.date, p.drug_name, NEW.drug_name
        FROM prescriptions p
        JOIN adverse_interactions ai
        ON ( (ai.drug_name_1 = p.drug_name AND ai.drug_name_2 = NEW.drug_name)
            OR (ai.drug_name_1 = NEW.drug_name AND ai.drug_name_2 = p.drug_name) )
        WHERE p.patient_id = NEW.patient_id
            AND p.drug_name <> NEW.drug_name;
    END;
    """
)

In [107]:
mycursor.execute(
    """
    SELECT TRIGGER_NAME, EVENT_MANIPULATION, EVENT_OBJECT_TABLE, ACTION_TIMING, ACTION_STATEMENT
    FROM information_schema.TRIGGERS
    WHERE TRIGGER_SCHEMA = 'hospital_project'
    """
)
result = mycursor.fetchone()
for r in result:
    print(r)

adverse_interaction_alert
INSERT
prescriptions
AFTER
BEGIN
        INSERT INTO alerts (patient_id, physician_id, alert_date, drug1, drug2)
        SELECT DISTINCT NEW.patient_id, NEW.physician_id, NEW.date, p.drug_name, NEW.drug_name
        FROM prescriptions p
        JOIN adverse_interactions ai
        ON ( (ai.drug_name_1 = p.drug_name AND ai.drug_name_2 = NEW.drug_name)
            OR (ai.drug_name_1 = NEW.drug_name AND ai.drug_name_2 = p.drug_name) )
        WHERE p.patient_id = NEW.patient_id
            AND p.drug_name <> NEW.drug_name;
    END


In [14]:
import pandas as pd

In [15]:
# load physician
data = pd.read_csv("physicians.csv")
for i,row in data.iterrows():
    sql = "INSERT INTO physicians VALUES (%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

('614-57-6885', 'Srinivasan', 'Cardiology', 4)
('702-16-8749', 'Wu', 'Dermatology', 10)
('571-13-9020', 'Mozart', 'Cardiology', 0)
('718-27-0905', 'Einstein', 'Psychiatry', 29)
('230-12-3219', 'El Said', 'Psychiatry', 12)
('163-50-5535', 'Gold', 'Neurology', 8)
('118-66-5958', 'Katz', 'Orthopedics', 3)
('156-28-1945', 'Singh', 'Orthopedics', 25)
('522-86-5827', 'Crick', 'Neurology', 0)
('510-55-9776', 'Brandt', 'Psychiatry', 25)
('460-35-6754', 'Kim', 'Orthopedics', 2)
('357-93-5814', 'Califieri', 'Cardiology', 21)


In [16]:
# load pharmacies
data = pd.read_csv("pharmacies.csv")
for i,row in data.iterrows():
    sql = "INSERT INTO pharmacies VALUES (%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

(1, ' Springfield Pharmacy', ' 123 Main St,  Springfield,  IL 62701', ' (217) 555-1234')
(2, ' Peachtree Meds', ' 456 Elm St,  Atlanta, GA 30303', ' (404) 555-5678')
(3, ' Lone Star Drugs', ' 789 Oak St,  Dallas, TX 75201', ' (214) 555-9101')
(4, ' Mile High Meds', ' 101 Pine St, Denver, CO80202', ' (303) 555-1121')
(5, ' Emerald City Pharmacy', ' 121 Spruce St,  Seattle,  WA 98101', ' (206) 555-1314')
(6, ' Golden Gate Drugs', ' 234 Market St, San Francisco, CA 94105', ' (415) 555-1515')
(7, ' Sunshine Pharmacy', ' 345 Palm Ave, Miami, FL 33101', ' (305) 555-1616')
(8, ' Liberty Meds', ' 567 Broadway St, New York, NY 10001', ' (212) 555-1717')
(9, ' Lakeside Drugs', ' 678 Lake Rd,  Minneapolis, MN 55401', ' (612) 555-1818')
(10, ' Desert Bloom Pharmacy', ' 890 Desert Blvd, Phoenix, AZ 85001', ' (602) 555-1919')
(11, ' Bayside Pharmacy', ' 112 Harbor Dr, San Diego, CA 92101', ' (619) 555-2020')
(12, ' Capital Meds', ' 345 Capitol St, Washington, DC 20001', ' (202) 555-2121')
(13, ' Win

In [18]:
# load drugs
data = pd.read_csv("drugs.csv")
for i,row in data.iterrows():
    sql = "INSERT INTO drugs VALUES (%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

(1, 'Primalovir')
(2, 'Olanzanafine')
(3, 'Avafoxin')
(4, 'Quixiposide')
(5, 'Cleotrana')
(6, 'Kanulin')
(7, 'Glucozepam Amcipentin')
(8, 'Divisporine Acetaclotide')
(9, 'Abnazole Toleluble')
(10, 'Dantopex Quixilinum')


In [19]:
# load patients
data = pd.read_csv("patients.csv")
data['birthdate'] = pd.to_datetime(data['birthdate']).dt.strftime('%Y-%m-%d')
for i,row in data.iterrows():
    sql = "INSERT INTO patients VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

('478-34-0781', 'Florance Saiz', '7 Fair Oaks Place', '1988-11-03', '614-57-6885')
('885-94-4721', 'Merry Di Pietro', '1 Old Shore Court', '1991-02-07', '702-16-8749')
('777-39-3296', 'Myron Cottem', '75875 Fulton Crossing', '1986-02-20', '718-27-0905')
('227-08-7452', 'Bearnard Remer', '18669 Heffernan Point', '2008-01-09', '230-12-3219')
('805-15-2755', 'Roxana Worster', '54 Hudson Junction', '1982-11-12', '163-50-5535')
('758-08-7274', 'Susanetta Petruska', '16276 Sutteridge Avenue', '1922-08-05', '118-66-5958')
('501-47-2038', 'Elvyn Rudinger', '48 Bowman Parkway', '2006-02-28', '156-28-1945')
('360-47-2098', 'Peter Lukasen', '552 Ryan Court', '1969-01-10', '522-86-5827')
('631-75-6048', 'Avrom Messer', '5030 Garrison Center', '1929-02-04', '510-55-9776')
('192-33-2887', 'Jacinda Stowe', '8 Colorado Alley', '1970-04-15', '357-93-5814')
('691-21-7304', 'Myrlene Yegoshin', '2 Sunnyside Court', '2001-06-03', '460-35-6754')
('303-13-5928', 'Krystyna Luckie', '54106 Barnett Plaza', '195

In [108]:
# load prescriptions
data = pd.read_csv("prescriptions.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
for i,row in data.iterrows():
    sql = "INSERT INTO prescriptions VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

(1, '478-34-0781', '614-57-6885', 'Avafoxin', '2023-03-11', 90)
(2, '758-08-7274', '118-66-5958', 'Cleotrana', '2023-03-12', 10)
(3, '758-08-7274', '118-66-5958', 'Primalovir', '2023-04-11', 20)
(4, '758-08-7274', '118-66-5958', 'Glucozepam Amcipentin', '2023-05-13', 12)
(5, '303-13-5928', '571-13-9020', 'Olanzanafine', '2023-05-24', 25)
(6, '303-13-5928', '571-13-9020', 'Primalovir', '2023-05-24', 16)
(7, '303-13-5928', '571-13-9020', 'Abnazole Toleluble', '2023-05-24', 5)
(8, '478-34-0781', '614-57-6885', 'Avafoxin', '2023-06-14', 60)
(9, '303-13-5928', '571-13-9020', 'Glucozepam Amcipentin', '2023-06-22', 3)
(10, '501-47-2038', '156-28-1945', 'Cleotrana', '2023-07-18', 20)
(11, '777-39-3296', '718-27-0905', 'Dantopex Quixilinum', '2023-08-02', 1)
(12, '501-47-2038', '156-28-1945', 'Cleotrana', '2023-08-21', 10)
(13, '478-34-0781', '614-57-6885', 'Avafoxin', '2023-09-17', 30)
(14, '478-34-0781', '614-57-6885', 'Quixiposide', '2023-09-17', 2)
(15, '501-47-2038', '156-28-1945', 'Avafox

In [96]:
# load adverse_interactions
data = pd.read_csv("adverse_reactions.csv")
for i,row in data.iterrows():
    sql = "INSERT INTO adverse_interactions VALUES (%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

('Cleotrana', 'Kanulin')
('Primalovir', 'Abnazole Toleluble')
('Primalovir', 'Olanzanafine')
('Olanzanafine', 'Glucozepam Amcipentin')
('Avafoxin', 'Kanulin')
('Quixiposide', 'Dantopex Quixilinum')
('Quixiposide', 'Avafoxin')
('Cleotrana', 'Avafoxin')
('Cleotrana', 'Quixiposide')


In [109]:
mycursor.execute("select * from alerts")
result = mycursor.fetchall()
for r in result:
    print(r)

('501-47-2038', '156-28-1945', datetime.date(2023, 9, 22), 'Avafoxin', 'Kanulin')
('501-47-2038', '156-28-1945', datetime.date(2023, 9, 22), 'Cleotrana', 'Avafoxin')
('501-47-2038', '156-28-1945', datetime.date(2023, 9, 22), 'Cleotrana', 'Kanulin')
('303-13-5928', '571-13-9020', datetime.date(2023, 5, 24), 'Olanzanafine', 'Primalovir')
('303-13-5928', '571-13-9020', datetime.date(2023, 5, 24), 'Primalovir', 'Abnazole Toleluble')
('303-13-5928', '571-13-9020', datetime.date(2023, 6, 22), 'Olanzanafine', 'Glucozepam Amcipentin')
('478-34-0781', '614-57-6885', datetime.date(2023, 9, 17), 'Avafoxin', 'Quixiposide')


In [112]:
# load pharmacy_fills
data = pd.read_csv("pharmacy_fills.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
for i,row in data.iterrows():
    sql = "INSERT INTO pharmacy_fills (pharmacy_id,prescription_id,date,cost) VALUES (%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

(1, 3, '2023-03-15', 60.53)
(3, 4, '2023-05-16', 41.5)
(1, 2, '2023-03-12', 18.0)
(8, 1, '2023-03-12', 46.53)
(10, 5, '2023-05-26', 47.5)
(7, 6, '2023-05-24', 92.1)
(5, 7, '2023-05-28', 41.65)
(5, 8, '2023-06-15', 94.6)
(11, 9, '2023-06-22', 31.0)
(2, 10, '2023-07-22', 14.55)
(12, 11, '2023-08-04', 92.0)
(14, 12, '2023-08-21', 42.85)
(6, 13, '2023-09-19', 31.65)
(7, 14, '2023-09-19', 11.0)
(9, 15, '2023-09-25', 46.8)
(4, 16, '2023-09-22', 42.75)


In [113]:
# load companies
data = pd.read_csv("companies.csv")
for i,row in data.iterrows():
    sql = "INSERT INTO companies VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

(1, 'Goodrx', '123 Main St, San Francisco, CA', '123-456-7890', 'Holly Jolly')
(2, 'PHARMASEE', '456 Elm St, New York, NY', '234-567-8901', 'Faker Maker')
(3, 'DRUGXO', '789 Maple St, Los Angeles, CA', '345-678-9012', 'Silly Putty')
(4, 'Pharmachoice', '101 Pine St, Chicago, IL', '456-789-0123', 'Connie Honey')
(5, 'Castox', '234 Oak St, Houston, TX', '567-890-1234', 'Laxmi Kant Sheth')
(6, 'Doktera', '567 Cedar St, Philadelphia, PA', '678-901-2345', 'I.P. Green')
(7, 'Lipdrugz', '890 Birch St, Phoenix, AZ', '789-012-3456', 'Boris Kotchakoff')
(8, 'Nurfarma', '123 Fir St, San Antonio, TX', '890-123-4567', 'Wu Liu')
(9, 'Munimed', '456 Redwood St, San Diego, CA', '901-234-5678', 'Kim Park')
(10, 'Arkmed', '789 Sequoia St, Dallas, TX', '012-345-6789', 'James Bond')


In [114]:
mycursor.execute("DESCRIBE contracts")
for x in mycursor:
    print(x)

('id', 'int', 'NO', 'PRI', None, '')
('company_id', 'int', 'YES', 'MUL', None, '')
('pharmacy_id', 'int', 'YES', 'MUL', None, '')
('drug_name', 'varchar(255)', 'YES', 'MUL', None, '')
('dosage', 'varchar(255)', 'YES', '', None, '')
('quantity', 'int', 'YES', '', None, '')
('date', 'date', 'YES', '', None, '')
('price', 'decimal(10,2)', 'YES', '', None, '')


In [115]:
# load contracts
data = pd.read_csv("contracts.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
for i,row in data.iterrows():
    sql = "INSERT INTO contracts (id,drug_name,dosage,pharmacy_id,company_id,quantity,date,price) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    print(tuple(row))
    mydb.commit()

(1, 'Cleotrana', 50, 5, 10, 40, '2023-10-03', 100.0)
(2, 'Primalovir', 500, 5, 10, 20, '2023-09-26', 40.5)
(3, 'Kanulin', 1000, 4, 9, 20, '2023-09-20', 10.5)
(4, 'Olanzanafine', 50, 6, 3, 80, '2023-09-24', 150.0)
(5, 'Avafoxin', 5, 15, 5, 30, '2023-09-29', 18.0)
(6, 'Quixiposide', 25, 5, 1, 30, '2023-10-02', 12.0)
(7, 'Glucozepam Amcipentin', 20, 10, 2, 80, '2023-09-27', 145.0)
(8, 'Divisporine Acetaclotide', 15, 12, 9, 30, '2023-10-02', 42.0)
(9, 'Abnazole Toleluble', 30, 14, 6, 40, '2023-09-21', 45.0)
(10, 'Dantopex Quixilinum', 100, 14, 8, 20, '2023-09-26', 20.0)
(11, 'Olanzanafine', 50, 1, 3, 90, '2023-09-30', 160.0)
(12, 'Olanzanafine', 75, 2, 3, 40, '2023-09-23', 60.0)
(13, 'Olanzanafine', 75, 3, 3, 20, '2023-09-23', 35.0)
(14, 'Glucozepam Amcipentin', 40, 8, 2, 20, '2023-10-02', 14.0)
(15, 'Divisporine Acetaclotide', 10, 9, 9, 40, '2023-09-28', 68.0)
(16, 'Dantopex Quixilinum', 100, 13, 8, 50, '2023-09-30', 50.0)
